## Lab Assignment 3 Scientific Computing

Nick Boon & Marleen Rijksen

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse.linalg import eigs, eigsh
from scipy.linalg import eig, eigh
from mpl_toolkits.mplot3d.axes3d import Axes3D

from matplotlib import animation, rc

from IPython.display import HTML
rc('animation', html='html5')

In [2]:
#need this for interactive 3d plotting!
%matplotlib notebook

In [3]:
#from https://stackoverflow.com/a/35126679
def set_aspect_equal_3d(ax):
    """
    Fix equal aspect bug for 3D plots in X and Y.    
    """

    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()

    xmean = np.mean(xlim)
    ymean = np.mean(ylim)

    plot_radius = np.max([
        abs(lim - mean_) for lims, mean_ in ((xlim, xmean), (ylim, ymean))
        for lim in lims
    ])

    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])

#From https://rajeshrinet.github.io/blog/2016/gray-scott/
def laplacian(shape):
    """
    Construct a sparse matrix that applies the 5-point discretization

    Input:
    shape - tuple of matrix shape (y,x)

    Output:
    Sparse matrix that evaluates the 5-point discretization of a shape-sized matrix
    """

    M, N = shape
    e = np.ones(N * M)
    e2 = ([1] * (N - 1) + [0]) * M
    e3 = ([0] + [1] * (N - 1)) * M

    A = spdiags([-4 * e, e2, e3, e, e], [0, -1, 1, -N, N], N * M, N * M)

    return A

def circular(r):
    """
    Fill matrix with circle of size r    
    """
    circle = np.zeros((r,r))
    for i,x in enumerate(np.arange(-r/2+0.5,r/2+0.5)):
        for j,y in enumerate(np.arange(-r/2+0.5,r/2+0.5)):
            if np.sqrt(x ** 2 + y ** 2) <= r/2:
                circle[j,i] = 1
    return circle

def laplacian_circle(r):
    """
    Creates the laplacian matrix from a matrix containing
    ones for sites part of the domain under consideration
    """
    circle = circular(r)
    
    self = circle.reshape(r**2)
    left = []
    right = []
    up = []
    down = []
    
    #bruteforce solution :(
    for j in range(r):
        for i in range(r):
            if i == 0:
                left.append(0)
            else:
                if circle[j,i-1] == 1 and circle[j,i] == 1:
                    left.append(1)
                else:
                    left.append(0)
            if i == r-1:
                right.append(0)
            else:
                if circle[j,i+1] == 1 and circle[j,i] == 1:
                    right.append(1)
                else:
                    right.append(0)
            if j == 0:
                up.append(0)
            else:
                if circle[j-1,i] == 1 and circle[j,i] == 1:
                    up.append(1)
                else:
                    up.append(0)
            if j == r-1:
                down.append(0)
            else:
                if circle[j+1,i] == 1 and circle[j,i] == 1:
                    down.append(1)
                else:
                    down.append(0)

    A = spdiags([-4 * self, left, right, up, down], [0, 1, -1, r, -r], r * r, r * r)
    return A

In [4]:
def show_eigenmodes(shape, eigenmodes, membrane_shape='square'):
    """
    Shows eigenmodes smalles eigenmodes. Can use 'square'-shaped
    or 'circle'-shaped membrane
    
    For circular shapes, the eigs()-function may return less
    eigenmodes. This seems to be random?
    """
    k = 5 * eigenmodes

    if membrane_shape == 'square':
        a = eigs(laplacian(shape), which='SM', k=k)
    elif membrane_shape == 'circle':
        a = eigs(laplacian_circle(shape[0]), which='SM', k=k)
    else:
        raise ValueError("Membrane shape can only be 'square' or 'circle'.")
    b = np.where(np.abs(a[0]) > 1E-6)

    eig_val = a[0][b]
    eig_vec = a[1][b]

    idx = np.abs(eig_val).argsort()
    idx = idx[0:eigenmodes]

    #create figure
    #every row has width 8 and height 3
    fig = plt.figure(figsize=(8, 3 * int((1 + len(idx)) / 2)))

    for i, ix in enumerate(idx):
        # set up the axes
        ax = fig.add_subplot(
            int((len(idx) + 1) / 2), 2, i + 1, projection='3d')

        # create surface
        x = np.arange(0, shape[1], 1)
        y = np.arange(0, shape[0], 1)
        x, y = np.meshgrid(x, y)

        z = a[1][:, ix].reshape(shape).real

        #frequency is 1/(c\lambda)=1/(c sqrt(-K))

        ax.plot_surface(x, y, z, rstride=3, cstride=3)
        ax.set_title("eigenvalue: %.2e\nfrequency: %.2f" %
                     (a[0][ix].real, np.sqrt(-a[0][ix].real)))
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.set_zlabel("amplitude")
        set_aspect_equal_3d(ax)
    fig.tight_layout()

def get_eigenfrequencies(shape,eigenmodes,membrane_shape):
    k = 5 * eigenmodes

    if membrane_shape == 'square':
        a = eigs(laplacian(shape), which='SM', k=k)
    elif membrane_shape == 'circle':
        a = eigs(laplacian_circle(shape[0]), which='SM', k=k)
    else:
        raise ValueError("Membrane shape can only be 'square' or 'circle'.")
    b = np.where(np.abs(a[0]) > 1E-6)

    eig_val = sorted(np.sqrt(-a[0][b][:eigenmodes].real))
    
    return eig_val

### Show eigenmodes in 3D plot

In [5]:
show_eigenmodes((100,100),10,'square')

<IPython.core.display.Javascript object>

In [6]:
show_eigenmodes((50,100),10,'square')

<IPython.core.display.Javascript object>

In [7]:
show_eigenmodes((100,100),10,'circle')

<IPython.core.display.Javascript object>

##### Show spectrum of eigenfrequencies depending on L

In [8]:
fig, ax = plt.subplots()
for i in np.arange(10,105,3):
    a = get_eigenfrequencies((i,i),10,'square')
    ax.scatter([i]*10,a)


<IPython.core.display.Javascript object>

In [9]:
fig, ax = plt.subplots()
for i in np.arange(10,105,3):
    a = get_eigenfrequencies((i,i),10,'circle')
    ax.scatter([i]*10,a)

<IPython.core.display.Javascript object>

##### Show animation for square domain

In [10]:
#TODO: Find better method for this

In [11]:
%%capture

shape = (20, 20)
eigm = 0
k = 25  #shape[0]*shape[1]-1
a = eigsh(laplacian(shape),which='SM', k=k)
b = np.where(np.abs(a[0])>1E-6)

idx = (np.abs(a[0][b])).argsort()

# create surface
x = np.arange(0, shape[1], 1)
y = np.arange(0, shape[0], 1)
x, y = np.meshgrid(x, y)

l = np.sqrt(-a[0][idx[eigm]])

A = a[1][:,idx[eigm]].reshape(shape).real

def animate(i):
    global A, t, x, y
    t += TIME_STEP
    T = A * np.cos(np.sqrt(-a[0][idx[eigm]]) * t)
    ax.clear()
    ax.plot_surface(x, y, T, rstride=1, cstride=1)
    ax.set_zlim3d(-0.1, 0.1)
    ax.set_title("Time: %.2f" % (t))
    set_aspect_equal_3d(ax)
    return ax

# set some parameters
FRAME_INTERVAL = 10
TIME_STEP = 0.1
NUM_FRAMES = int(2 * np.pi / (l * TIME_STEP))
t = 0

fig = plt.figure(figsize=(6, 6))
# set up the axes
ax = fig.add_subplot(1, 1, 1, projection='3d')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("amplitude")
ani = animation.FuncAnimation(
    fig, animate, frames=NUM_FRAMES, interval=FRAME_INTERVAL, blit=True)

In [12]:
ani

##### Show animation for circular domain

In [13]:
%%capture

shape = (20, 20)
eigm = 0
k = 25
a = eigsh(laplacian_circle(shape[0]),which='SM', k=k)
b = np.where(np.abs(a[0])>1E-6)

idx = (np.abs(a[0][b])).argsort()

# create surface
x = np.arange(0, shape[1], 1)
y = np.arange(0, shape[0], 1)
x, y = np.meshgrid(x, y)

l = np.sqrt(-a[0][idx[eigm]])

A = a[1][:,idx[eigm]].reshape(shape).real

def animate(i):
    global A, t, x, y
    t += TIME_STEP
    T = A * np.cos(np.sqrt(-a[0][idx[eigm]]) * t)
    ax.clear()
    ax.plot_surface(x, y, T, rstride=1, cstride=1)
    ax.set_zlim3d(-0.1, 0.1)
    ax.set_title("Time: %.2f" % (t))
    set_aspect_equal_3d(ax)
    return ax

# set some parameters
FRAME_INTERVAL = 10
TIME_STEP = 0.1
NUM_FRAMES = int(2 * np.pi / (l * TIME_STEP))
t = 0

fig = plt.figure(figsize=(6, 6))
# set up the axes
ax = fig.add_subplot(1, 1, 1, projection='3d')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("amplitude")
ani = animation.FuncAnimation(
    fig, animate, frames=NUM_FRAMES, interval=FRAME_INTERVAL, blit=True)

In [ ]:
ani